# 올림픽 메달과 국가별 GDP관계 분석
## Content
1896~2012년까지의 올림픽 데이터
31,165개의 데이터

## Acknowledgements
Data was provided by the IOC Research and Reference Service and published by The Guardian's Datablog.

## What is GDP?
GDP란, Gross Domestic Product의 약자로서 국내의 총생산을 의미한다.

per capita는 1인당이므로, GDP per Capita 는 1인당 국내 총생산을 의미.

## data
출처 : https://www.kaggle.com/the-guardian/olympic-games 

summer.csv : 1896~2014년까지의 올림픽 메달 기록

Year : 올림픽 개최년도
City : 올림픽 개최도시
Sport : 종목 대분류
Discipline : 종목 중분류
Athlete : 선수
Country : 국적
Gender : 성별
Event : 종목 소분류
Medal : 금/은/동

dictionary.csv : 2014년의 인구 및 GDP data

Country : 나라이름
Code : 올림픽 국가코드
Population : 인구
GDP per Capita : 일인당 국내 총생산

### by = BW


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#하계 올림픽 파일 읽기
olympic = pd.read_csv('summer.csv', encoding='utf-8')
olympic.head()

In [ ]:
#olympic data 정보 확인
olympic.info()

In [ ]:
#olympic 칼럼 확인
olympic.columns

In [ ]:
#olympic 칼럼 순서 변경
olympic = olympic[['Country', 'Year', 'Sport', 'Discipline', 'Athlete', 'City', 'Gender',
       'Event', 'Medal']]
olympic.head()

In [ ]:
#Gedner로 그룹화하여 count
gender_count = olympic.groupby('Gender').count()
gender_count.head()

In [ ]:
#Country로 그룹화한 후 메달 개수 순서로 정렬
country = olympic.groupby('Country').count()
country = country.sort_values(by='Medal',ascending=False)
country.head()

In [ ]:
#상위 30개 국가만 good_country로 생성
good_country = country.head(30)
good_country['Medal'].sort_values().plot(kind='barh', grid=True, figsize=(10,10))
plt.xlabel('Medal')
plt.show()

In [ ]:
#선수들의 메달 갯수 확인해보기
olympic_athlete = olympic.groupby('Athlete').count()
olympic_athlete.sort_values(by='Medal', ascending = False).head()

In [ ]:
#마이클펠프스의 기록 확인
olympic.loc[olympic['Athlete'] == 'PHELPS, Michael']

In [ ]:
#한국 올림픽 메달 기록 확인
korea = olympic.loc[olympic['Country'] == 'KOR']
korea.head()

In [ ]:
korea.info()

In [ ]:
#한국 메달 가장 많이 딴 선수 확인해보기
korea_medal = korea.groupby('Athlete').count()
korea_medal.sort_values(by='Medal', ascending=False).head()

In [ ]:
#한국에서 메달을 가장많이 확보한 종목
korea['Discipline'].value_counts().plot(kind='barh', grid=True, figsize=(10,10), )
plt.xlabel('Medal')
plt.show()

## 핸드볼이 가장 많다? -> 단체 종목이기 때문

In [ ]:
#단체종목일 경우 가장 첫번째 레코드만 남겨두고 삭제 by drop_duplicates()
korea_discipline = korea.drop_duplicates(subset=['Year', 'Event', 'Medal'])
korea_discipline.info()

In [ ]:
korea_discipline['Discipline'].value_counts().plot(kind='barh', grid=True, figsize=(10,10))
plt.xlabel('Medal')
plt.title('KOREA OLYMPIC MEDAL')
plt.show()

In [ ]:
#다시 실습내용 진행하기 위해 
olympic.head()

In [ ]:
#마찬가지로 단체 종목을 메달 중복 제거한 olympic_count DF 생성
olympic_count = olympic.drop_duplicates(subset=['Year', 'Country', 'Medal', 'Event'])
olympic_count.info()

In [ ]:
#index reset
olympic_count = olympic_count.reset_index(drop=True)
olympic_count.tail()

In [ ]:
#size를 통해 country로 그룹한후 Year로 그룹핑
country_medal_count = olympic_count.groupby(['Country','Year']).size()
#size의 결과는 series형태이므로 to_frame()을 통해 Data Frame형태로 바꿔줌.
country_medal_count = country_medal_count.to_frame().reset_index()

In [ ]:
country_medal_count.head()

In [ ]:
#column rename하기
country_medal_count = country_medal_count.rename(columns = {0 : 'Medal'})
country_medal_count.head()

In [ ]:
#pivot_table(index, column, value, aggfunc, margins)
country_medal_year = country_medal_count.pivot_table(index = "Country", columns = "Year", values = "Medal",aggfunc = 'sum', margins = True)
country_medal_year.head()

In [ ]:
country_medal_year.columns[ : -4]

In [ ]:
#2000년 이전 기록 다 삭제
a = country_medal_year.columns[:-4]
for i in a :
    del country_medal_year[i]
country_medal_year.head() 

In [ ]:
#상위 30개 국가만 진행
top_country = country_medal_year.sort_values(by='All', ascending = False).head(40)

In [ ]:
#2000년 이전 메달 개수 다 합침
top_country = top_country.fillna(0)
#NaN수를 모두 0으로 대입
top_country['before_2000'] = top_country['All'] - top_country[2004] - top_country[2008] - top_country[2012]
top_country

In [ ]:
#GDP 파일 열기
gdp = pd.read_csv('dictionary.csv')
gdp.head()

In [ ]:
gdp.rename(columns = {'Country' : 'Fullname', 'Code' : 'Country'}, inplace = True)

In [ ]:
gdp.head()

In [ ]:
#데이터프레임 합치기 
result = pd.merge(top_country, gdp, on='Country')
result

In [ ]:
result = result[['Country','Fullname', 'before_2000', 2004, 2008, 2012, 'All', 'Population',
       'GDP per Capita']]
result.head()

In [ ]:
#최근 메달 증가율
result['Growth'] = (result[2004] + result[2008]+ result[2012]) / result['before_2000'] * 100

result.sort_values(by='Growth', ascending=False)

In [ ]:
result = result.fillna(7815)

## 러시아와 중국의 증가율의 강세가 보임. 한국도 눈에 띈다.
### 러시아는 이전 소비에트 연방 이전의 결과가 반영되지 못해서 강세가 두드러지는 것으로 보임.

In [ ]:
result.set_index('Country', inplace=True)

In [ ]:
result.head()

## 상관계수 구해보기

 -1에 가까운 값이 얻어지면 : 누가 봐도 매우 강력한 음(-)의 상관. 오히려 너무 확고하기 때문에 일부 연구자들은 데이터를 조작한 게 아닌가 의심할 정도이다. 사회과학 한정이다. 순수학문에 가까운 분야일수록 요구되는 상관관계는 높은 편.

 -0.5 정도의 값이 얻어지면 : 강력한 음(-)의 상관. 연구자는 변인 x 가 증가하면 변인 y 가 감소한다고 자신 있게 말할 수 있다.

 -0.2 정도의 값이 얻어지면 : 음(-)의 상관이긴 한데 너무 약해서 모호하다. 상관관계가 없다고는 할 수 없지만 좀 더 의심해 봐야 한다.

 0 정도의 값이 얻어지면 : 대부분의 경우, 상관관계가 존재하지 않을 것이라고 간주된다. 다른 후속 연구들을 통해 뒤집어질지는 모르지만 일단은 회의적이다. 2차 방정식 그래프와 비슷한 모양이 될 경우 상관관계는 있으나 상관계수는 0에 가깝게 나온다.

 0.2 정도의 값이 얻어지면 : 너무 약해서 의심스러운 양(+)의 상관. 이것만으로는 상관관계에 대해 아주 장담할 수는 없다. 하지만 사회과학에선 매우 큰 상관관계가 있는 것으로 간주한다.

 0.5 정도의 값이 얻어지면 : 강력한 양(+)의 상관. 변인 x 가 증가하면 변인 y 가 증가한다는 주장은 이제 통계적으로 지지받고 있다.

 1에 가까운 값이 얻어지면 : 이상할 정도로 강력한 양(+)의 상관. 위와 마찬가지로, 이렇게까지 확고한 상관관계는 오히려 쉽게 찾아보기 어렵다

In [ ]:
#총 메달 개수와 인구와의 관계
np.corrcoef(result['All'], result['Population'])

In [ ]:
#메달 증가율과 인구와의 관계
np.corrcoef(result['Growth'], result['Population'])

In [ ]:
#총 메달 개수와 GDP의 관계
np.corrcoef(result['All'], result['GDP per Capita'])

In [ ]:
#메달 증가율과 GDP의 관계
np.corrcoef(result['Growth'], result['GDP per Capita'])

In [ ]:
#platform -> font한글 폰트하기 위해서 

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)

else :
    print('Unknown system.. sorry~~')

## 시각화

In [ ]:
result['All'].plot(kind='barh', grid=True, figsize=(10,10))
plt.xlabel('Medal')
plt.title('Country Medal')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(result['All'], result['Population'], s= 50)
plt.xlabel('메달 수')
plt.ylabel('인구 수')
plt.title('메달수와 인구수와의 관계')
plt.grid()
plt.show()


In [ ]:
result['rate_pop'] = result['Population'] / result['Population'].sum()
result

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(result['Growth'], result['Population'], s= 50)
plt.xlabel('메달 증가율')
plt.ylabel('인구 수')
plt.title('메달증가율과 인구 수와의 관계')
plt.grid()
plt.show()

In [ ]:
#너무 큰 차이나와 러시아 빼보자
result_except_big = result.drop(['CHN','RUS'],0)

In [ ]:
result_except_big

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(result_except_big['Growth'], result_except_big['Population'], s= 50)
plt.xlabel('메달 증가율')
plt.ylabel('인구 수')
plt.title('메달 증가율과 인구 수와의 관계')
plt.grid()
plt.show()

In [ ]:
np.corrcoef(result_except_big['Growth'], result_except_big['Population'])

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(result['Growth'], result['GDP per Capita'], s= 50)
plt.xlabel('메달 증가율')
plt.ylabel('GDP')
plt.title('메달 증가율과 GDP와의 관계')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(result['All'], result['GDP per Capita'], s= 50)
plt.xlabel('메달 수')
plt.ylabel('GDP')
plt.title('메달수와 GDP와의 관계')
plt.grid()
plt.show()

In [ ]:
fp1 = np.polyfit(result['All'], result['GDP per Capita'], 1)
f1 = np.poly1d(fp1)
fx = np.linspace(100, 3000, 100)
plt.figure(figsize=(10,10))
plt.scatter(result['All'], result['GDP per Capita'], s= 50)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='g')
plt.xlabel('메달 수')
plt.ylabel('GDP per Capita')
plt.title('메달 수와 GDP의 관계')
plt.grid()
plt.show()

In [ ]:
fp1 = np.polyfit(result['Growth'], result['GDP per Capita'], 1)
f1 = np.poly1d(fp1)
fx = np.linspace(0, 200, 100)
plt.figure(figsize=(10,10))
plt.scatter(result['Growth'], result['GDP per Capita'], s= 50)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='g')
plt.xlabel('메달 증가율')
plt.ylabel('GDP per Capita')
plt.title('메달증가율과 GDP의 관계')
plt.grid()
plt.show()

In [ ]:
fp1 = np.polyfit(result['Growth'], result['GDP per Capita'], 1)

f1 = np.poly1d(fp1)
fx = np.linspace(0,200,100)

result['오차'] = np.abs(result['GDP per Capita'] - f1(result['Growth']))

result_sort = result.sort_values(by='오차', ascending=False)
result_sort.head()

In [ ]:
plt.figure(figsize=(14,10))
plt.scatter(result['Growth'], result['GDP per Capita'],
           c=result['오차'], s = 50)
plt.plot(fx, f1(fx), ls='dashed', lw =3 , color='g')

for n in range(10) :
    plt.text(result_sort['Growth'][n]*1.02, result_sort['GDP per Capita'][n]*0.98,
            result_sort.index[n], fontsize=15)

plt.xlabel('메달 증가율')
plt.ylabel('GDP per Capita')

plt.colorbar()
plt.grid()
plt.show()